In [ ]:
import pandas as pd
import sys
import os
import pickle
import warnings

# On ignore les warnings "invalid value encountered in divide" qui polluent tes logs
warnings.filterwarnings("ignore")

from evidently.report import Report
from evidently.metric_preset import DataDriftPreset, DataQualityPreset
from evidently.test_suite import TestSuite
from evidently.tests import TestNumberOfColumnsWithMissingValues, TestShareOfDriftedColumns
from evidently.ui.workspace.remote import RemoteWorkspace

# ==========================================
# 1. CONFIGURATION & NETTOYAGE (LE FIX EST ICI)
# ==========================================
EVIDENTLY_TOKEN = os.environ.get("EVIDENTLY_TOKEN")
raw_project_id = os.environ.get("EVIDENTLY_PROJECT_ID", "")

# 🧹 NETTOYAGE : On enlève les guillemets (" ou ') et les espaces vides
EVIDENTLY_PROJECT_ID = str(raw_project_id).strip().replace('"', '').replace("'", "")

print(f"ℹ️ Project ID utilisé (nettoyé) : {EVIDENTLY_PROJECT_ID}")

ARTIFACTS_PATH = "../backend/src/processors" 

if not EVIDENTLY_TOKEN or not EVIDENTLY_PROJECT_ID:
    print("❌ ERREUR : Token ou Project ID manquant.")
    sys.exit(1)

# ==========================================
# 2. CHARGEMENT DES DONNÉES
# ==========================================
print("📂 Chargement des données...")

try:
    # Adapte ce chemin si nécessaire (ex: ../../backend/src/processors)
    # selon où tu lances le script
    with open(os.path.join(ARTIFACTS_PATH, "preprocessed_data.pkl"), "rb") as f:
        data = pickle.load(f)
    
    with open(os.path.join(ARTIFACTS_PATH, "features_config.pkl"), "rb") as f:
        config = pickle.load(f)
    
    columns = config['final_feature_order']
    
    reference_data = pd.DataFrame(data['X_train_scaled'], columns=columns)
    reference_data['target'] = data['y_train'] 
    
    current_data = pd.DataFrame(data['X_test_scaled'], columns=columns)
    current_data['target'] = data['y_test']
    
    print(f"   ✅ Reference: {reference_data.shape}")
    print(f"   ✅ Current: {current_data.shape}")

except Exception as e:
    print(f"❌ Erreur chargement données : {e}")
    # En local pour tester, tu peux commenter sys.exit(1) si tu veux juste tester la connexion cloud
    sys.exit(1)

# ==========================================
# 3. DASHBOARD CLOUD
# ==========================================
print("☁️ Connexion à Evidently Cloud...")

try:
    ws = RemoteWorkspace("https://app.evidently.cloud", EVIDENTLY_TOKEN)
    
    # On vérifie que l'ID est valide pour Evidently
    # ws.search_project(EVIDENTLY_PROJECT_ID) # Optionnel
    
    print("📊 Génération du rapport...")
    report = Report(metrics=[
        DataDriftPreset(),
        DataQualityPreset()
    ])
    
    report.run(reference_data=reference_data, current_data=current_data)
    
    print("⬆️ Envoi vers le Cloud...")
    ws.add_report(EVIDENTLY_PROJECT_ID, report)
    print("✅ Rapport envoyé avec succès !")

except ValueError as ve:
    print(f"❌ ERREUR UUID : L'ID du projet est toujours invalide : {EVIDENTLY_PROJECT_ID}")
    print("👉 Vérifie ta variable d'environnement, elle ne doit contenir que des chiffres et des lettres (a-f).")
    print(f"Détail : {ve}")
except Exception as e:
    print(f"⚠️ Erreur Cloud (Autre) : {e}")

# ==========================================
# 4. QUALITY GATE
# ==========================================
print("🛡️ Exécution des Tests...")

tests = TestSuite(tests=[
    TestShareOfDriftedColumns(lt=0.5), 
    TestNumberOfColumnsWithMissingValues(eq=0) 
])

tests.run(reference_data=reference_data, current_data=current_data)
test_results = tests.as_dict()

if not test_results['summary']['all_passed']:
    print("❌ ECHEC : Data Drift critique !")
    sys.exit(1)
else:
    print("✅ SUCCÈS : Données stables.")
    sys.exit(0)

ℹ️ Project ID utilisé (nettoyé) : 01f16019b2950-f65e-7167-a3f4-67232238fbfd
📂 Chargement des données...
   ✅ Reference: (319110, 18)
   ✅ Current: (79778, 18)
☁️ Connexion à Evidently Cloud...
📊 Génération du rapport...


In [26]:
import sys
import os
import pickle
import pandas as pd
import numpy as np

from evidently.report import Report
from evidently.metric_preset import DataDriftPreset
from evidently.test_suite import TestSuite
from evidently.tests import TestShareOfDriftedColumns

# ==========================================
# 1. CONFIGURATION ET CHEMINS
# ==========================================
current_dir = os.getcwd()  # Répertoire courant

# Chemin vers backend/src pour importer feature_store.py
BACKEND_SRC_PATH = os.path.abspath(os.path.join(current_dir, '..', 'backend', 'src'))
sys.path.insert(0, BACKEND_SRC_PATH)

# Chemin vers les processeurs (.pkl)
ARTIFACTS_PATH = os.path.abspath(os.path.join(BACKEND_SRC_PATH, 'processors'))

# Chemin vers les nouvelles données brutes (Simulation de prod)
# On suppose que le fichier est dans data/crime_v1.csv à la racine du projet
NEW_DATA_PATH = os.path.abspath(os.path.join(current_dir,'..', 'data', 'last_rows.csv'))

# Import du Feature Store (La même logique que l'API !)
try:
    from feature_store import CrimeFeatureStore
except ImportError:
    print("❌ Impossible d'importer CrimeFeatureStore. Vérifiez que backend/src/feature_store.py existe.")
    sys.exit(1)

def get_reference_data():
    """Charge les données d'entraînement (Reference) depuis le pickle"""
    print("📦 Chargement des données de référence (Train)...")
    data_path = os.path.join(ARTIFACTS_PATH, "preprocessed_data.pkl")
    
    if not os.path.exists(data_path):
        print(f"❌ Fichier manquant : {data_path}")
        sys.exit(1)

    with open(data_path, "rb") as f:
        data = pickle.load(f)
    
    # On a besoin des noms de colonnes pour Evidently
    # On peut les récupérer via le Feature Store ou le config pickle
    with open(os.path.join(ARTIFACTS_PATH, "features_config.pkl"), "rb") as f:
        config = pickle.load(f)
        
    cols = config['final_feature_order']
    
    # Reconstruction du DataFrame Reference
    ref_df = pd.DataFrame(data['X_train_scaled'], columns=cols)
    # Optionnel : Ajouter la target si on veut monitorer le Concept Drift
    # ref_df['target'] = data['y_train'] 
    
    return ref_df

def process_current_data_with_store(df_raw, store):
    """
    Transforme les nouvelles données brutes en utilisant le Feature Store.
    Simule ce qui se passe dans l'API.
    """
    print("⚙️ Transformation des nouvelles données via Feature Store...")
    
    processed_rows = []
    
    # On itère sur chaque ligne comme si c'était une requête API unique
    # C'est un peu plus lent que du batch, mais ça garantit 100% de cohérence avec l'API
    input_records = df_raw.to_dict(orient='records')
    
    for record in input_records:
        # get_online_features renvoie un tableau numpy (1, N)
        vector = store.get_online_features(record)
        processed_rows.append(vector[0]) # On prend la première ligne (le vecteur plat)
    
    # On recrée un DataFrame avec les bonnes colonnes
    current_df = pd.DataFrame(processed_rows, columns=store.required_features)
    
    return current_df

def run_production_monitoring():
    try:
        # 1. Initialisation du Feature Store
        store = CrimeFeatureStore(processors_path=ARTIFACTS_PATH)
        store.load_artifacts()
        
        # 2. Récupération de la Référence
        reference_data = get_reference_data()
        
        # 3. Récupération et Traitement du Current (Nouveau)
        if not os.path.exists(NEW_DATA_PATH):
            print(f"⚠️ Fichier de nouvelles données introuvable : {NEW_DATA_PATH}")
            print("   -> Création d'un dataset factice pour le test...")
            # Si pas de fichier, on prend un échantillon du train pour tester le script
            current_data = reference_data.sample(100)
        else:
            print(f"📂 Lecture des nouvelles données : {NEW_DATA_PATH}")
            # On prend un échantillon aléatoire de 500 lignes
            df_new_raw = pd.read_csv(NEW_DATA_PATH).sample(500)
            
            # Transformation via le Store
            current_data = process_current_data_with_store(df_new_raw, store)
        
        print(f"📊 Comparaison : Ref {reference_data.shape} vs New {current_data.shape}")

        # 4. Analyse du Drift avec Evidently
        print("🚀 Analyse du Drift en cours...")
        
        drift_suite = TestSuite(tests=[
            TestShareOfDriftedColumns(lt=0.3) # Alerte si > 30% des colonnes dévient
        ])
        
        drift_suite.run(reference_data=reference_data, current_data=current_data)
        
        # Sauvegarde
        report_path = "production_drift_report.html"
        drift_suite.save_html(report_path)
        print(f"✅ Rapport généré : {os.path.abspath(report_path)}")
        
        # 5. Verdict
        results = drift_suite.as_dict()
        if not results['summary']['all_passed']:
            print("🚨 ALERTE : Le modèle dérive ! (Data Drift detected)")
            sys.exit(1)
        else:
            print("✅ Tout va bien. Pas de dérive significative.")
            sys.exit(0)

    except Exception as e:
        print(f"❌ Erreur critique : {e}")
        import traceback
        traceback.print_exc()
        sys.exit(1)

if __name__ == "__main__":
    run_production_monitoring()


✅ Feature Store: Artifacts loaded.
📦 Chargement des données de référence (Train)...
⚠️ Fichier de nouvelles données introuvable : c:\Users\asus\Desktop\mlops\MLOPS\data\last_rows.csv
   -> Création d'un dataset factice pour le test...
📊 Comparaison : Ref (319110, 17) vs New (100, 17)
🚀 Analyse du Drift en cours...


c:\Users\asus\Desktop\mlops\MLOPS\venv\lib\site-packages\numpy\lib\_function_base_impl.py:2922: RuntimeWarning:

invalid value encountered in divide

c:\Users\asus\Desktop\mlops\MLOPS\venv\lib\site-packages\numpy\lib\_function_base_impl.py:2923: RuntimeWarning:

invalid value encountered in divide

c:\Users\asus\Desktop\mlops\MLOPS\venv\lib\site-packages\numpy\lib\_function_base_impl.py:2922: RuntimeWarning:

invalid value encountered in divide

c:\Users\asus\Desktop\mlops\MLOPS\venv\lib\site-packages\numpy\lib\_function_base_impl.py:2923: RuntimeWarning:

invalid value encountered in divide



✅ Rapport généré : c:\Users\asus\Desktop\mlops\MLOPS\monitoring\production_drift_report.html
✅ Tout va bien. Pas de dérive significative.


SystemExit: 0

c:\Users\asus\Desktop\mlops\MLOPS\venv\lib\site-packages\IPython\core\interactiveshell.py:3558: UserWarning:

To exit: use 'exit', 'quit', or Ctrl-D.



Pourquoi cela a-t-il échoué ?
C'est tout à fait normal dans votre contexte de test :
Vous avez utilisé un petit échantillon (probablement 100 ou 500 lignes).
Sur un petit échantillon, les statistiques bougent beaucoup par rapport au gros fichier d'entraînement (300k lignes).
Evidently a détecté ces différences statistiques légitimes.

In [28]:
import pandas as pd

# Chemin du fichier source
input_file = r"C:\Users\asus\Desktop\mlops\data\data.csv"

# Charger le CSV
data = pd.read_csv(input_file)

# Prendre les 5 dernières lignes
last_rows1 = data.tail(400)

# Chemin du fichier de sortie
output_file = r"C:\Users\asus\Desktop\mlops\data\crime_last5.csv"

# Sauvegarder dans un nouveau CSV
last_rows1.to_csv(output_file, index=False)

print(f"✅ Les 300 dernières lignes ont été enregistrées dans {output_file}")


✅ Les 300 dernières lignes ont été enregistrées dans C:\Users\asus\Desktop\mlops\data\crime_last5.csv
